In [31]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import eli5
from eli5.lime import TextExplainer
import sys

import warnings
warnings.filterwarnings("ignore")

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ParquetReaderTesting").getOrCreate()

In [3]:
df = spark.read.parquet("./data/")

In [4]:
df.columns

['clipTarget',
 'clip_id',
 'owner_id',
 'clipContentCategory',
 'date',
 'contentAudioEmbedding',
 'contentEmbedding',
 'AUTOTAG_max_proba_sum',
 'AUTOTAG_most_frequent',
 'AUTOTAG_top_50_precentage_frames',
 'description',
 'asr',
 'confidence_asr']

In [14]:
df2 = df.select("clip_id", "description", "clipTarget").toPandas()

In [15]:
df2 = df2.dropna(subset=['description'])

In [16]:
TARGET = 'clipTarget'

In [55]:
def simple_preprocessing(data):
    stop = stopwords.words('russian')
    data = data.str.replace('[^\w\s]','')
    data  = data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    data = data.str.replace('[^\w\s]',' ')
    data = data.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    return data

In [56]:
df2['clean_text'] = simple_preprocessing(df2.description)

In [57]:
df2.shape

(266124, 4)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(df2['clean_text'], df2[TARGET], 
                                                    test_size=0.3, 
                                                    random_state=2022)

In [64]:
log_reg = LogisticRegression()
vectorizer = CountVectorizer()

In [65]:
X_train_text = vectorizer.fit_transform(X_train)
X_test_text = vectorizer.transform(X_test)

In [66]:
pipe = Pipeline([('clf', log_reg)])

In [67]:
%%time
pipe.fit(X_train_text, y_train)

CPU times: user 6min 20s, sys: 25.3 s, total: 6min 46s
Wall time: 3min 11s


Pipeline(steps=[('clf', LogisticRegression())])

In [68]:
print(roc_auc_score(y_test, pipe.predict_proba(X_test_text), multi_class='ovr'))
print(f1_score(y_test, pipe.predict(X_test_text), average='macro'))

0.8595233569187006
0.3841095697867601


In [72]:
print(classification_report(y_test, pipe.predict(X_test_text)))

                   precision    recall  f1-score   support

          ANIMALS       0.56      0.54      0.55      3757
              ART       0.27      0.05      0.08       258
        BACKSTAGE       0.52      0.35      0.42      1065
           BEAUTY       0.63      0.64      0.63      3765
             BLOG       0.15      0.17      0.16      4032
            BOOKS       0.73      0.57      0.64       447
         BUSINESS       0.40      0.34      0.36       580
        CELEBRITY       0.47      0.20      0.28       235
       CHALLENGES       0.54      0.55      0.54       348
         COMMERCE       0.17      0.04      0.07       660
          COSPLAY       0.69      0.35      0.46       529
         CREATION       0.41      0.22      0.29      1610
          CULTURE       0.45      0.33      0.38      1336
            DANCE       0.56      0.46      0.51      2807
          DRAWING       0.68      0.59      0.63      1705
        EDUCATION       0.52      0.30      0.38      1

In [73]:
eli5.show_weights(estimator=pipe, feature_names= list(vectorizer.get_feature_names()), top=(50, 5))

In [97]:
ROW_NUMB = 18
print(f"text: {X_test.iloc[ROW_NUMB]}")
print(f"true: {y_test.iloc[ROW_NUMB]}")
print(f"pred: {pipe.predict(X_test_text)[ROW_NUMB]}")

text: сезон близко мотоцикл мотосезон мотомосква
true: VEHICLES
pred: VEHICLES


In [98]:
from eli5.lime import TextExplainer

eli5.show_prediction(log_reg, X_test.iloc[ROW_NUMB], vec=vectorizer)